In [1]:
from RAG import Retrieval_Augmented_Generation

In [2]:
rag = Retrieval_Augmented_Generation()

In [3]:
db = rag.VectorDatabase()

/home/junaid-ul-hassan/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


ValidationError: 1 validation error for HuggingFaceEmbeddings
device
  extra fields not permitted (type=value_error.extra)

34612